In [1]:
import os
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

In [2]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

In [3]:
download_train = drive.CreateFile({'id': '16ldCVmTIM95z1-m3FQZe1Kj-HrEKe7fJ'})

In [4]:
download_train.GetContentFile('train_LbELtWX.zip')
!unzip train_LbELtWX.zip

Streaming output truncated to the last 5000 lines.
  inflating: train_LbELtWX/train/11621.png  
  inflating: __MACOSX/train_LbELtWX/train/._11621.png  
  inflating: train_LbELtWX/train/57674.png  
  inflating: __MACOSX/train_LbELtWX/train/._57674.png  
  inflating: train_LbELtWX/train/30837.png  
  inflating: __MACOSX/train_LbELtWX/train/._30837.png  
  inflating: train_LbELtWX/train/48754.png  
  inflating: __MACOSX/train_LbELtWX/train/._48754.png  
  inflating: train_LbELtWX/train/59881.png  
  inflating: __MACOSX/train_LbELtWX/train/._59881.png  
  inflating: train_LbELtWX/train/4409.png  
  inflating: __MACOSX/train_LbELtWX/train/._4409.png  
  inflating: train_LbELtWX/train/47467.png  
  inflating: __MACOSX/train_LbELtWX/train/._47467.png  
  inflating: train_LbELtWX/train/51211.png  
  inflating: __MACOSX/train_LbELtWX/train/._51211.png  
  inflating: train_LbELtWX/train/17244.png  
  inflating: __MACOSX/train_LbELtWX/train/._17244.png  
  inflating: train_LbELtWX/train/37194.png

In [5]:
import keras
from tqdm import tqdm
import pandas as pd
import numpy as np
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Activation
from keras.layers import Conv2D, MaxPooling2D, Convolution1D, GlobalMaxPooling2D
from keras.utils import to_categorical
from sklearn.model_selection import train_test_split
from keras import backend as K
from keras.models import load_model
import tensorflow as tf
from tensorflow import keras
import IPython

!pip install -U keras-tuner
import kerastuner as kt


     |████████████████████████████████| 71kB 5.9MB/s 
  Created wheel for keras-tuner: filename=keras_tuner-1.0.2-cp36-none-any.whl size=78939 sha256=55a561d48d100a452d90d0d34e2060994cd14d5290b14d298c28a71384d27aeb
  Stored in directory: /root/.cache/pip/wheels/bb/a1/8a/7c3de0efb3707a1701b36ebbfdbc4e67aedf6d4943a1f463d6
  Created wheel for terminaltables: filename=terminaltables-3.1.0-cp36-none-any.whl size=15358 sha256=7931509674c6c74d6b75f07193d10b433b9f515196a2e61794051010b69972db
  Stored in directory: /root/.cache/pip/wheels/30/6b/50/6c75775b681fb36cdfac7f19799888ef9d8813aff9e379663e
Successfully built keras-tuner terminaltables


In [6]:
train_dataset = pd.read_csv("train_LbELtWX/train.csv", dtype=str);
train_dir = ('train_LbELtWX/train');

train_dataset["id"] = train_dataset["id"].apply(lambda x : '{}{}'.format(x, '.png'));

train_df=train_dataset[0:50000]
test_df=train_dataset[50000:]

train_df

,id,label
0,1.png,9
1,2.png,0
2,3.png,0
3,4.png,3
4,5.png,0
...,...,...
49995,49996.png,6
49996,49997.png,6
49997,49998.png,5
49998,49999.png,1


In [7]:
X_train, X_test, y_train, y_test = train_test_split(train_df["id"], train_df["label"], random_state=42, test_size=0.2)

df_train = pd.concat([X_train, y_train], axis=1);
df_test = pd.concat([X_test, y_test], axis=1);

df_test

,id,label
33553,33554.png,5
9427,9428.png,7
199,200.png,2
12447,12448.png,9
39489,39490.png,0
...,...,...
28567,28568.png,4
25079,25080.png,0
18707,18708.png,9
15200,15201.png,9


In [8]:
image_width, image_height = 30, 30;
epochs = 10;
batch_size = 20;

if K.image_data_format() == "channels_first":
    input_shape = (3, image_width, image_height);
else:
    input_shape = (image_width, image_height, 3);
    
train_datagen = ImageDataGenerator(
    rescale = 1.0/255,
    shear_range = 0.2,
    zoom_range = 0.2,
    horizontal_flip = True
);

test_datagen = ImageDataGenerator(rescale = 1.0/255);


In [9]:
train_generator = train_datagen.flow_from_dataframe(
    dataframe = df_train,
    directory = "train_LbELtWX/train/",
    x_col="id", 
    y_col="label", 
    target_size = (image_width, image_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode="rgb",
    seed=42
    #classes=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]  
);

validation_generator = test_datagen.flow_from_dataframe(
    dataframe = df_test,
    directory = "train_LbELtWX/train/",
    x_col="id", 
    y_col="label", 
    target_size = (image_width, image_height),
    batch_size = batch_size,
    class_mode = 'categorical',
    color_mode="rgb",
    seed=42
    #classes=["0", "1", "2", "3", "4", "5", "6", "7", "8", "9"]  
);

Found 40000 validated image filenames belonging to 10 classes.
Found 10000 validated image filenames belonging to 10 classes.


In [10]:
#Find Best Model
########################################

from tqdm import tqdm;

train_img = []
for i in tqdm(range(1,train_df.shape[0] + 1)):
    img = image.load_img('train_LbELtWX/train/'+str(i)+'.png', target_size=(28,28,1), color_mode = 'grayscale')
    img = image.img_to_array(img)
    img = img/255.0
    train_img.append(img)

train_images = np.array(train_img)

train_labels = train_df['label'].astype(int).values
#y = to_categorical(train_labels)

train_labels


100%|██████████| 50000/50000 [00:10<00:00, 4681.48it/s]


array([9, 0, 0, ..., 5, 1, 7])

In [11]:
def build_model(hp):
  model = keras.Sequential([
              keras.layers.Conv2D(
                    filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
                    kernel_size = hp.Choice('conv_1_kernel', values = [3,5]),
                    activation='relu',
                    input_shape=(28,28,1)
                    ),

              keras.layers.Conv2D(
                    filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
                    kernel_size = hp.Choice('conv_2_kernel', values = [3,5]),
                    activation='relu'
                    ),
                  
              keras.layers.Conv2D(
                    filters=hp.Int('conv_3_filter', min_value=32, max_value=64, step=16),
                    kernel_size = hp.Choice('conv_3_kernel', values = [3,5]),
                    activation='relu'
                    ),
              
              keras.layers.Conv2D(
                    filters=hp.Int('conv_4_filter', min_value=32, max_value=64, step=16),
                    kernel_size = hp.Choice('conv_4_kernel', values = [3,5]),
                    activation='relu'
                    ),
                  
              keras.layers.Flatten(),
              keras.layers.Dense(
                  units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
                  activation='relu'
                    ),
              
              keras.layers.Dense(10, activation='softmax')
              ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
                loss='sparse_categorical_crossentropy',
                metrics=['accuracy'])
  
  return model


In [12]:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

tuner_search = RandomSearch(build_model, objective='val_accuracy', max_trials=5)

In [13]:
tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.2)

Trial 5 Complete [00h 00m 21s]
val_accuracy: 0.09929999709129333

Best val_accuracy So Far: 0.8984000086784363
Total elapsed time: 00h 01m 50s
INFO:tensorflow:Oracle triggered exit


In [14]:
model = tuner_search.get_best_models(num_models=1)[0]

model.summary()


#Use Best Model Parameters.
########################################

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 112)       1120      
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 32)        89632     
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 20, 20, 48)        13872     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 18, 18, 48)        20784     
_________________________________________________________________
flatten (Flatten)            (None, 15552)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                995392    
_________________________________________________________________
dense_1 (Dense)              (None, 10)                6

In [15]:
model = Sequential()
model.add(Conv2D(48, (3, 3), input_shape = input_shape))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Conv2D(64, (3, 3)));
model.add(Activation('relu'));
model.add(MaxPooling2D(pool_size=(2, 2)));

model.add(Conv2D(48, (3, 3)));
model.add(Activation('relu'));

model.add(Conv2D(64, (3, 3)));
model.add(Activation('relu'));
model.add(MaxPooling2D(pool_size=(2, 2)));
model.add(Dropout(0.25));

model.add(Flatten());
model.add(Dense(64));
model.add(Activation('relu'));
model.add(Dropout(0.5));

model.add(Dense(10));
model.add(Activation('softmax'));

model.summary()


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 28, 28, 48)        1344      
_________________________________________________________________
activation (Activation)      (None, 28, 28, 48)        0         
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 48)        0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 12, 12, 64)        27712     
_________________________________________________________________
activation_1 (Activation)    (None, 12, 12, 64)        0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 6, 6, 64)          0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 4, 4, 48)         

In [16]:
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6);

model.compile(optimizer = opt,
              loss='categorical_crossentropy',
              #optimizer = 'rmsprop',
              metrics = ['accuracy']);


In [17]:
STEP_SIZE_TRAIN=train_generator.n//train_generator.batch_size
STEP_SIZE_VALID=validation_generator.n//validation_generator.batch_size

model.fit(train_generator,
          steps_per_epoch=STEP_SIZE_TRAIN,
          validation_data=validation_generator,
          validation_steps=STEP_SIZE_VALID,
          epochs=epochs);

Epoch 1/10
2000/2000 [==============================] - 33s 16ms/step - loss: 1.8119 - accuracy: 0.3511 - val_loss: 0.7940 - val_accuracy: 0.7236
Epoch 2/10
2000/2000 [==============================] - 33s 16ms/step - loss: 1.0172 - accuracy: 0.6415 - val_loss: 0.6485 - val_accuracy: 0.7624
Epoch 3/10
2000/2000 [==============================] - 32s 16ms/step - loss: 0.8509 - accuracy: 0.6936 - val_loss: 0.5746 - val_accuracy: 0.7754
Epoch 4/10
2000/2000 [==============================] - 32s 16ms/step - loss: 0.7465 - accuracy: 0.7284 - val_loss: 0.5436 - val_accuracy: 0.7987
Epoch 5/10
2000/2000 [==============================] - 32s 16ms/step - loss: 0.6973 - accuracy: 0.7453 - val_loss: 0.5179 - val_accuracy: 0.8045
Epoch 6/10
2000/2000 [==============================] - 32s 16ms/step - loss: 0.6557 - accuracy: 0.7613 - val_loss: 0.4946 - val_accuracy: 0.8083
Epoch 7/10
2000/2000 [==============================] - 32s 16ms/step - loss: 0.6227 - accuracy: 0.7737 - val_loss: 0.4959 -

In [18]:
model.save_weights('apparel_prediction.h5');

In [19]:
# making predictions

test_generator = test_datagen.flow_from_dataframe(
    dataframe = test_df,
    directory = "train_LbELtWX/train/",
    x_col="id", 
    target_size = (image_width, image_height),
    batch_size = batch_size,
    class_mode = None,
    color_mode = "rgb",
    seed=42,
    shuffle=False
)


Found 10000 validated image filenames.


In [20]:
test_generator.reset()

predictions=model.predict(test_generator,verbose=1,steps=test_generator.n/batch_size);

predictions = np.argmax(predictions, axis=1);

filenames=test_generator.filenames;

results=pd.DataFrame({"id":filenames,
                      "Predictions":predictions});

results["Predictions"]=results["Predictions"].astype('float32')

results["Expected"]=test_df.reset_index().drop(columns=['index'])["label"].astype('float32')

results

500/500 [==============================] - 3s 6ms/step


,id,Predictions,Expected
0,50001.png,9.0,9.0
1,50002.png,2.0,2.0
2,50003.png,1.0,1.0
3,50004.png,0.0,0.0
4,50005.png,2.0,2.0
...,...,...,...
9995,59996.png,5.0,5.0
9996,59997.png,1.0,1.0
9997,59998.png,3.0,3.0
9998,59999.png,0.0,0.0


In [21]:
Prediction_Percentage = len(results[results["Predictions"]==results["Expected"]])*100/len(results)

Prediction_Percentage

84.08

In [22]:
download_test = drive.CreateFile({'id': '1d6buKxF5ioKx5BV3m9__XVSeTLQjCkmh'})

In [23]:
download_test.GetContentFile('test_ScVgIM0.zip')
!unzip test_ScVgIM0.zip

Streaming output truncated to the last 5000 lines.
  inflating: test_ScVgIM0/test/64654.png  
  inflating: __MACOSX/test_ScVgIM0/test/._64654.png  
  inflating: test_ScVgIM0/test/62225.png  
  inflating: __MACOSX/test_ScVgIM0/test/._62225.png  
  inflating: test_ScVgIM0/test/66043.png  
  inflating: __MACOSX/test_ScVgIM0/test/._66043.png  
  inflating: test_ScVgIM0/test/69370.png  
  inflating: __MACOSX/test_ScVgIM0/test/._69370.png  
  inflating: test_ScVgIM0/test/60432.png  
  inflating: __MACOSX/test_ScVgIM0/test/._60432.png  
  inflating: test_ScVgIM0/test/68708.png  
  inflating: __MACOSX/test_ScVgIM0/test/._68708.png  
  inflating: test_ScVgIM0/test/60354.png  
  inflating: __MACOSX/test_ScVgIM0/test/._60354.png  
  inflating: test_ScVgIM0/test/66725.png  
  inflating: __MACOSX/test_ScVgIM0/test/._66725.png  
  inflating: test_ScVgIM0/test/69416.png  
  inflating: __MACOSX/test_ScVgIM0/test/._69416.png  
  inflating: test_ScVgIM0/test/63885.png  
  inflating: __MACOSX/test_ScVgIM

In [24]:
test = pd.read_csv('test_ScVgIM0/test.csv')

test["id"] = test["id"].apply(lambda x : '{}{}'.format(x,".png") )


In [25]:
# making predictions

test_generator = test_datagen.flow_from_dataframe(
    dataframe = test,
    directory = "test_ScVgIM0/test/",
    x_col="id", 
    target_size = (image_width, image_height),
    batch_size = batch_size,
    class_mode = None,
    color_mode = "rgb",
    seed=42,
    shuffle=False
)


Found 10000 validated image filenames.


In [26]:
test_generator.reset()

predictions=model.predict(test_generator,verbose=1,steps=test_generator.n/batch_size);

predictions = np.argmax(predictions, axis=1);

filenames=test_generator.filenames;

results=pd.DataFrame({"id":filenames,
                      "label":predictions});

results


500/500 [==============================] - 3s 5ms/step


,id,label
0,60001.png,9
1,60002.png,2
2,60003.png,1
3,60004.png,1
4,60005.png,6
...,...,...
9995,69996.png,9
9996,69997.png,1
9997,69998.png,8
9998,69999.png,1
